In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread("D:/c data/cvimg/sudoku.png")
img = cv2.resize(img,(400,400))

gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)          #Hough Line Transform works best on single-channel images.
edg = cv2.Canny(gry,20,250)                         #Uses Canny Edge Detector.,Thresholds are 20 (low) and 250 (high).

lines = cv2.HoughLines(edg,1,np.pi/180,200)         #1 → resolution of rho (distance from origin to line) = 1 pixel.,np.pi/180 → resolution of theta (angle step) = 1 degree.

for r,th in lines[0]:
    a = np.cos(th)
    b = np.sin(th)

    x0 = a*r
    y0 = b*r

    x1 = int(x0+1000*(-b))                      #1000? → It’s just a large number, ensures the line extends outside the image size.
    y1 = int(x0+1000*(a))
    x2 = int(x0-1000*(-b))
    y2 = int(x0-1000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow("org",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# shortest method

In [3]:
img = cv2.imread("D:/c data/cvimg/sudoku.png")
img = cv2.resize(img,(400,400))

gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)          
edg = cv2.Canny(gry,20,250)

l = cv2.HoughLinesP(edg,1,np.pi/180,200,minLineLength = 180,maxLineGap = 100)

for i in l:
    x1,y1,x2,y2 = i[0]
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv2.imshow("org",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Template Matching 

In [4]:
img = cv2.imread("D:/c data/cvimg/group.jpeg")
gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

tmp = cv2.imread("D:/c data/cvimg/group_copy1.jpeg")
gry1= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

w,h = gry1.shape[1],gry1.shape[0]

res = cv2.matchTemplate(gry,gry1,cv2.TM_CCOEFF_NORMED)

thr = 0.99

l = np.where(res>=thr)

for i in zip(*l[::-1]):
    cv2.rectangle(img,i,(i[0]+w,i[1]+h),(0,0,255),2)

#print(gry1.shape)
#cv2.imshow("res",res)
img = cv2.resize(img,(600,600))
cv2.imshow("org",img)
cv2.imshow("tmp",tmp)
cv2.waitKey(0)
cv2.destroyAllWindows()

# hough circle

In [5]:
img = cv2.imread("D:/c data/cvimg/ball.png")
gry = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gry = cv2.medianBlur(gry,5)

c = cv2.HoughCircles(gry,cv2.HOUGH_GRADIENT,1,20,param1 = 90 ,param2 = 80,minRadius = 0,maxRadius=0)

data = np.uint16(np.around(c))

for (x,y,r) in data [0,:]:
    cv2.circle(img,(x,y),r,(0,0,255),2)

cv2.imshow("org",gry)
cv2.imshow("Circle",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Grabcut algo for background change

In [9]:
img = cv2.imread("D:/c data/cvimg/grabcut.jfif")

mask1 = np.zeros(img.shape[:2],np.uint8)

bgmask = np.zeros((1,65),np.float64)*255                               # background mask
fgmask = np.zeros((1,65),np.float64)*255                               # foreground mask

# [x1,y1,x2,y2]
r = [21,63,267,168]
cv2.grabCut(img,mask1,r,bgmask,fgmask,10,cv2.GC_INIT_WITH_RECT)        #cv2.GC_INIT_WITH_RECT = tells GrabCut to initialize with rectangle

mask2 = np.where((mask1 == 2 )|(mask1 == 0),0,1).astype("uint8")       #(mask1 == 2) -> creates a Boolean array (True where pixel is 2, else False).

                                                                        #(mask1 == 0) → Boolean array (True where pixel is 0).

                                                                        # | (bitwise OR) → combines the two conditions.
img = img*mask2[:,:,np.newaxis]

cv2.imshow("org",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# vedio bg removal by using algo

In [13]:
cap = cv2.VideoCapture("D:/c data/cvimg/Screen Recording 2025-07-31 224915.mp4")

algo1 = cv2.createBackgroundSubtractorKNN()
algo2 = cv2.createBackgroundSubtractorMOG2()

while True:
    r,f = cap.read()
    if r == True:
        f = cv2.resize(f,(500,400))
        r1 = algo1.apply(f)
        r2 = algo2.apply(f)
        cv2.imshow("algo1",r1)
        cv2.imshow("algo2",r2)
        cv2.imshow("org",f)
        if cv2.waitKey(40) & 0xff == ord("p"):
             break
    else:
        break

cap.release()
cv2.destroyAllWindows()